In [4]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.2 MB/s eta 0:00:00


In [8]:
# Re-import required packages after kernel reset
import numpy as np
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

# -------------------------------
# Forecasting Model: Simple Exponential Smoothing
# -------------------------------
class ExponentialSmoothing:
    def __init__(self, alpha):
        """
        Initialize the smoothing factor.
        alpha: Smoothing constant between 0 and 1
        """
        self.alpha = alpha
        self.forecast_values = []  # Store computed forecast values

    def forecast(self, data):
        """
        Perform forecasting using the SES formula:
        Ft = α * Y(t−1) + (1 − α) * F(t−1)
        data: List of historical values (e.g., past demand)
        returns: List of forecasted values
        """
        forecast = [data[0]]  # Start with the first actual data point as the initial forecast
        for t in range(1, len(data)):
            # Apply SES formula using previous forecast and actual value
            next_val = self.alpha * data[t - 1] + (1 - self.alpha) * forecast[-1]
            forecast.append(next_val)
        self.forecast_values = forecast  # Save the forecast values
        return forecast

# -------------------------------
# Optimization Model: Transportation Problem
# -------------------------------
class TransportationModel:
    def __init__(self, supply, demand, cost_matrix):
        """
        Initializes the transportation problem with input data.
        supply: List of available quantities at each source
        demand: List of required quantities at each destination
        cost_matrix: 2D list of per-unit transportation costs from each source to each destination
        """
        self.supply = supply
        self.demand = demand
        self.cost = cost_matrix
        self.num_sources = len(supply)
        self.num_destinations = len(demand)

    def solve(self):
        """
        Solves the transportation problem to minimize total cost while meeting supply and demand constraints.
        returns: Dictionary containing status, total cost, and allocation matrix
        """
        # Create the LP problem with the objective to minimize cost
        model = LpProblem("Transportation_Problem", LpMinimize)

        # Define decision variables x[i][j]: amount shipped from source i to destination j
        x = [[LpVariable(f"x_{i}_{j}", lowBound=0)
              for j in range(self.num_destinations)]
              for i in range(self.num_sources)]

        # Objective function: Minimize total shipping cost
        model += lpSum(self.cost[i][j] * x[i][j]
                       for i in range(self.num_sources)
                       for j in range(self.num_destinations))

        # Constraint: Total shipments from each source should not exceed its supply
        for i in range(self.num_sources):
            model += lpSum(x[i][j] for j in range(self.num_destinations)) <= self.supply[i]

        # Constraint: Total received at each destination must meet its demand
        for j in range(self.num_destinations):
            model += lpSum(x[i][j] for i in range(self.num_sources)) >= self.demand[j]

        # Solve the LP problem
        model.solve()

        # Prepare results to return
        result = {
            "Status": LpStatus[model.status],  # Optimal, Infeasible, etc.
            "Total Cost": value(model.objective),  # Total minimized cost
            "Allocations": [[x[i][j].varValue for j in range(self.num_destinations)]
                            for i in range(self.num_sources)]  # Allocation matrix
        }
        return result

# -------------------------------
# Sample Execution of the Module
# -------------------------------

# Historical demand data for forecasting
time_series_data = [100, 110, 120, 130, 125, 140]

# Initialize forecasting model with smoothing factor alpha = 0.5
forecast_model = ExponentialSmoothing(alpha=0.5)
forecasted_values = forecast_model.forecast(time_series_data)  # Generate forecasts

# Supply at each source
supply = [20, 30, 25]

# Demand at each destination
demand = [10, 30, 15, 20]

# Cost matrix: cost[i][j] = cost from source i to destination j
cost_matrix = [
    [8, 6, 10, 9],
    [9, 7, 4, 2],
    [3, 4, 2, 5]
]

# Initialize transportation model
transport_model = TransportationModel(supply, demand, cost_matrix)
optimization_result = transport_model.solve()  # Solve transportation problem

# Display transportation allocations as a readable DataFrame
allocation_df = pd.DataFrame(optimization_result["Allocations"])
print("\nTransportation Allocation Matrix:")
print(allocation_df)


# Return key results: forecasts, status, and total cost
{
    "Forecasted Values": forecasted_values,
    "Transportation Status": optimization_result["Status"],
    "Total Transportation Cost": optimization_result["Total Cost"]
}



Transportation Allocation Matrix:
      0     1     2     3
0   0.0  20.0   0.0   0.0
1   0.0   0.0  10.0  20.0
2  10.0  10.0   5.0   0.0


{'Forecasted Values': [100, 100.0, 105.0, 112.5, 121.25, 123.125],
 'Transportation Status': 'Optimal',
 'Total Transportation Cost': 280.0}